In [2]:
import pandas as pd
import numpy as np

Определить вероятность того, что в группе, состоящей из 23 человек, у двух людей будет совпадение дней рождения(число и месяц).

In [4]:
bd = pd.Series(range(1, 366))

In [8]:
t = bd.sample(23, replace=True)

In [21]:
rooms = [bd.sample(100, replace=True).duplicated().max() for _ in range(10000)]

In [22]:
np.mean(rooms)

np.float64(1.0)

In [47]:
from math import factorial

1 - factorial(365)/(pow(365, 23)*factorial(365-23))

0.5072972343239854

Экзамен проходит по следующей схеме:
Если некоторый билет уже был вытянут, то после ответа экзаменатор откладывает его в сторону. Студент выучил 20 билетов из 30. Когда ему выгоднее идти, первым или вторым, чтобы вероятность вытянуть выученный билет была больше?

In [53]:
print(20/30, (19/29 + 20/29)/ 2)

0.6666666666666666 0.6724137931034483


In [52]:
20/29

0.6896551724137931

In [96]:
from random import shuffle

n = 10000
tickets = list(range(1, 31))
student = list(range(1, 21))
result = []
second_result = []

for i in range(n):
    shuffle(tickets)
    result.append(tickets[0] in student)
    second_result.append(tickets[1] in student)

In [97]:
np.mean(result)

np.float64(0.6702)

In [98]:
np.mean(second_result)

np.float64(0.6697)

Ночью произошло ДТП с участием такси. В городе работают две компании такси - "Синие" и "Зеленые". "Зеленым" принадлежит 85% такси, "Синим" 15%. Свидетель аварии утверждает, что такси принадлежало "Синим". Следственный эксперимент показал, что ночью свидетель верно определяет цвет таки в 80%. Какова вероятность того, что такси действительно принадлежало "Синим"?

$$P(H1/A) = \frac{P(H1)*P(A/H1)}{P(A)}$$

- $P(H1/A)$ - вероятность того, что такси является синим при условии, что свидетель сказал, что оно синее
- $P(H1)$ - вероятность того, что такси является синим
- $P(A/H1)$ - вероятность того, что свидетель сказал, что такси синее, когда такси действительно синее
- $P(A)$ - вероятность того, что свидетель скажет, что свидетель назовет виновыми в аварии синих

In [131]:
p_h1_a = 0.15
p_h1 = 0.8
p_a = 0.15 * 0.8 + 0.85 * 0.2
p_h1_a * p_h1 / p_a

0.4137931034482758

Проверим с помощью monte carlo. \
С помощью random binomial создадим вероятность появления синего такси:

In [108]:
np.random.binomial(1, 0.15)

1

In [109]:
def witness(taxi):
    if np.random.binomial(1, 0.8):
        return taxi
    return 1 - taxi

Тогда 0 - зеленое такси, 1 - синее такси

In [112]:
witness(0)


1

In [119]:
n = 100000
result = []
for _ in range(n):
    taxi = np.random.binomial(1, 0.15)
    witness_answer = witness(taxi)
    result.append((taxi, witness_answer))

In [120]:
df = pd.DataFrame(result, columns=['taxi', 'witness_answer'])

In [121]:
df.groupby('witness_answer')['taxi'].mean()

witness_answer
0   0.042
1   0.410
Name: taxi, dtype: float64

Русская рулетка. Есть револьвер, с 2 вставленными подряд патронами. Первый крутит барабан, стреляет и остается жив. Твоя очередь. Тебе предлагается покрутить барабан перед выстрелом. Согласишься или сразу выстрелишь и почему?

In [134]:
chamber = [1, 1, 0, 0, 0, 0]

In [135]:
def one_turn(chamber):
    new_chamber = [0, 0, 0, 0, 0, 0]
    n = len(chamber)
    for i in range(n):
        if i < n-1:
            new_chamber[i+1] = chamber[i]
        else:
            new_chamber[0] = chamber[i]
    return new_chamber

In [137]:
one_turn(chamber)

[0, 1, 1, 0, 0, 0]

In [138]:
def spin_chamber(chamber):
    n = np.random.randint(1, 7)
    for i in range(n):
        chamber = one_turn(chamber)
    return chamber

In [148]:
pd.Series([spin_chamber(chamber) for i in range(10000)]).value_counts(normalize=True)

[0, 0, 0, 1, 1, 0]   0.170
[1, 1, 0, 0, 0, 0]   0.169
[0, 0, 1, 1, 0, 0]   0.167
[0, 0, 0, 0, 1, 1]   0.166
[1, 0, 0, 0, 0, 1]   0.165
[0, 1, 1, 0, 0, 0]   0.163
Name: proportion, dtype: float64

In [149]:
# Заряжаем револьвер
chamber = [1, 1, 0, 0, 0, 0]
# Первый игрок крутит барабан
chamber = spin_chamber(chamber)
# Первый игрок стреляет
chamber = one_turn(chamber)
p1 = chamber[0]
chamber[0] = 0
# Второй игрок вращает барабан -- нужно ли это действие?
chamber = spin_chamber(chamber)
# Второй игрок стреляет
chamber = one_turn(chamber)
p2 = chamber[0]

In [152]:
n = 10000
result = []

for _ in range(n):
    # Заряжаем револьвер
    chamber = [1, 1, 0, 0, 0, 0]
    # Первый игрок крутит барабан
    chamber = spin_chamber(chamber)
    # Первый игрок стреляет
    chamber = one_turn(chamber)
    p1 = chamber[0]
    if p1 == 1:
        continue

    chamber_wt = one_turn(chamber)
    p2_wt = chamber_wt[0]
    # Второй игрок вращает барабан -- нужно ли это действие?
    chamber = spin_chamber(chamber)
    # Второй игрок стреляет
    chamber = one_turn(chamber)
    p2 = chamber[0]

    result.append((p2_wt, p2))

In [156]:
df = pd.DataFrame(result, columns=['p2_wt', 'p2'])

In [158]:
print(df['p2_wt'].mean(), df['p2'].mean())

0.24746509919177076 0.3252020573108009


In [ ]:
6 * pow(1/3, )

In [163]:
factorial(6)/(factorial(4)*factorial(2))

15.0